In [176]:
from google.colab import drive  #to use google drive
import os
import time
from IPython.display import clear_output 
import random
from datetime import date
import pandas as pd
import numpy as np
from numpy import unique
from numpy import argmax
from sklearn.metrics import accuracy_score
from keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow import keras
import os.path
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import copy

In [187]:
#1 piedra, 2 papel, 3 tijera

class Cachipun:
    def __init__(self,file='/content/drive/MyDrive/Piedra_Papel_Tijera/piedra_papel_tijera_historico.csv', file_train='/content/drive/MyDrive/Piedra_Papel_Tijera/piedra_papel_tijera_train.csv',file_ANN='/content/drive/MyDrive/Piedra_Papel_Tijera/ANN_Cachipun.h5'):
        self.file=file
        self.file_columns = ['DATE','piedra_j1','papel_j1','tijera_j1','piedra_m1','papel_m1','tijera_m1','piedra_j2','papel_j2','tijera_j2','piedra_m2','papel_m2','tijera_m2','piedra_j3','papel_j3','tijera_j3','piedra_m3','papel_m3','tijera_m3','piedra_j4','papel_j4','tijera_j4','piedra_m4','papel_m4','tijera_m4','piedra_j5','papel_j5','tijera_j5','piedra_m5','papel_m5','tijera_m5']
        self.file_ANN=file_ANN
        self.file_train=file_train
        self.file_train_columns = ['piedra_j1','papel_j1','tijera_j1','piedra_m1','papel_m1','tijera_m1','piedra_j2','papel_j2','tijera_j2','piedra_m2','papel_m2','tijera_m2','piedra_j3','papel_j3','tijera_j3','piedra_m3','papel_m3','tijera_m3','piedra_j4','papel_j4','tijera_j4','piedra_m4','papel_m4','tijera_m4','piedra_j5','papel_j5','tijera_j5','piedra_m5','papel_m5','tijera_m5','NEXT_MOVE_PIEDRA','NEXT_MOVE_PAPEL','NEXT_MOVE_TIJERA']
        drive.mount("/content/drive")

    def get_machine_move(self,current_moves):
        self.current_moves=current_moves
        if os.path.isfile(self.file_ANN):
            model = keras.models.load_model(self.file_ANN)
            
            self.current_moves.pop(0)

            for i in range(len(self.current_moves),24):
                self.current_moves.append(0)
            
            my_array = np.array(self.current_moves)
             
            my_array=my_array.reshape((1, len(my_array)))

            yhat = model.predict(my_array)
            yhat = argmax(yhat, axis=-1).astype('int')

            #le red predice lo que voy a sacar
            if yhat==0: #tijera
                opcion_maquina=3
                #lo que le gana a tijeras es piedra
                opcion_maquina=1
            elif yhat==1: #papel
                opcion_maquina=2
                #lo que le gana a papel es tijeras
                opcion_maquina=3
            elif yhat==2: #piedra 1
                opcion_maquina=1
                #lo que le gana a piedra es papel
                opcion_maquina=2

        else:
            opcion_maquina = random.randint(1, 3)
            print("no hay red neuronal")
        
        return opcion_maquina

    def train_ann(self):
        dataframe = pd.read_csv(self.file_train, sep=";", usecols=self.file_train_columns)

        #elimina las filas que sean 'NEXT_MOVE_PIEDRA','NEXT_MOVE_PAPEL','NEXT_MOVE_TIJERA' igual a -1
        dataframe=dataframe[(dataframe['NEXT_MOVE_PIEDRA']!=-1) & (dataframe['NEXT_MOVE_PAPEL']!=-1) &  (dataframe['NEXT_MOVE_TIJERA']!=-1)]

        #creo nueva columnas con resultados
        dataframe["resultado"] = dataframe['NEXT_MOVE_PIEDRA'] * 100 + dataframe['NEXT_MOVE_PAPEL'] * 10 +  dataframe['NEXT_MOVE_TIJERA']

        #elimina las ultima jugada #columnas-> 'piedra_j5','papel_j5','tijera_j5','piedra_m5','papel_m5','tijera_m5'
        dataframe.drop(columns=['piedra_j5','papel_j5','tijera_j5','piedra_m5','papel_m5','tijera_m5','NEXT_MOVE_PIEDRA','NEXT_MOVE_PAPEL','NEXT_MOVE_TIJERA'], inplace=True)


        df = dataframe.values

        # split into input (X) and output (y) variables
        X, y = df[:, 0:-1], df[:, -1:]

        #y transforma el resultado en 1,2,3

        n_features = X.shape[1]


        # encode strings to integer
        y = LabelEncoder().fit_transform(y)

        n_class = len(unique(y))
        # split data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

        if os.path.isfile(self.file_ANN):
            model = keras.models.load_model(self.file_ANN) 
            
        else:
            # define the keras model
            model = Sequential()
            model.add(Dense(100, input_dim=n_features, activation='relu', kernel_initializer='he_normal'))
            model.add(Dropout(0.1))
            model.add(Dense(200, activation='relu', kernel_initializer='he_normal'))
            model.add(Dropout(0.1))
            model.add(Dense(50, activation='relu', kernel_initializer='he_normal'))
            model.add(Dropout(0.1))
            model.add(Dense(30, activation='relu', kernel_initializer='he_normal'))
            model.add(Dropout(0.1))
            model.add(Dense(n_class, activation='softmax'))
            # compile the keras model
            model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')



        # fit the keras model on the dataset
        model.fit(X_train, y_train, epochs=500, batch_size=50, verbose=0)
        # evaluate on test set
        
        yhat = model.predict(X_test)
        yhat = argmax(yhat, axis=-1).astype('int')
        acc = accuracy_score(y_test, yhat)
        print('Accuracy: %.3f' % acc)

        # Guardar el Modelo
        model.save(self.file_ANN)
    
    def processing_file(self):
        #vacia el archivo base y lo lleva al archivo que entrenara la red neuronal
        df_final = pd.DataFrame()
        df = pd.read_csv(self.file, sep=";", usecols=self.file_columns)
        df=df.drop('DATE', axis=1)
        for i in range(0,df.shape[0]): 
            list_row_i_df=df.iloc[i].values.tolist()            
            for j in range(0,6):
                zeros =  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
                if j==5:
                    zeros = list_row_i_df[0:j*6]+zeros[j*6:30]+[-1,-1,-1]
                else:
                    zeros = list_row_i_df[0:j*6]+zeros[j*6:30]+list_row_i_df[j*6:j*6+3]
                
                df_output = pd.DataFrame([zeros], columns = self.file_train_columns)
                df_final = pd.concat([df_final,df_output],ignore_index=True)
            df_final.to_csv(self.file_train, index=None, mode="a", sep=';', header=False)
            
        df_rewrite=pd.DataFrame(columns = self.file_columns)
        df_rewrite.to_csv(self.file ,sep=";", index = False)
            
        
    def __transformar(self,opcion):
        jugada=list()
        if opcion==1:
            jugada=[1,0,0]
        elif opcion==2:
            jugada=[0,1,0]
        else:
            jugada=[0,0,1]
        return jugada

    def __definir_punto(self,opcion_jugador, opcion_maquina):
        punto=list()
        if opcion_jugador==opcion_maquina:
            punto=[0,0]
        elif opcion_jugador == 1 and opcion_maquina==2:
            punto=[0,1]
        elif opcion_jugador == 1 and opcion_maquina==3:
            punto=[1,0]
        elif opcion_jugador == 2 and opcion_maquina==1:
            punto=[1,0]
        elif opcion_jugador == 2 and opcion_maquina==3:
            punto=[0,1]
        elif opcion_jugador == 3 and opcion_maquina==1:
            punto=[0,1]
        elif opcion_jugador == 3 and opcion_maquina==2:
            punto=[1,0]
        return punto[0],punto[1]

    def main(self):

        print('Bienvenido, al juego de Piedra Papel o Tijera, mejor conocido en Chile como Ca Chi Pun')
        time.sleep(0.4)
        print('Vamos a lo que has venido')
        time.sleep(0.4)
        print('A perder XD')
        time.sleep(0.4)
        print('La regla es simple, cuando diga \'CHI\' deberas escribir 1 para Piedra, 2 para papel y 3 para tijera')
        time.sleep(0.4)
        print('Cualquier otra cosa la interpretare como Piedra. PARTAMOOOOSSS')
        time.sleep(2.4)
        clear_output()
 

        while True:
            puntaje_humano=0
            puntaje_maquina=0
            CSV_row=list()
            CSV_row.append(date.today())
            for i in range(1,6):
                clear_output()
                print('HUMANO\t\tMAQUINA')
                print(str(puntaje_humano)+'\t\t'+str(puntaje_maquina)+'\n')
                print('CA ', end='')
                time.sleep(1.0)

                clear_output()
                print('HUMANO\t\tMAQUINA')
                print(str(puntaje_humano)+'\t\t'+str(puntaje_maquina)+'\n')
                opcion_jugador = input('CHIII (ahora escribe 1,2 o 3): ')
                time.sleep(1.0)

                clear_output()
                print('HUMANO\t\tMAQUINA')
                print(str(puntaje_humano)+'\t\t'+str(puntaje_maquina)+'\n')
                print('PUUUUNNN ', end='')
                time.sleep(2.0)
    
                ##CALCULO
                if opcion_jugador=='1' or opcion_jugador=='2':
                    opcion_jugador=int(opcion_jugador)
                else:
                    opcion_jugador=3

                CSV_row2= copy.copy(CSV_row)
                
                opcion_maquina = self.get_machine_move(CSV_row2)


                aux1,aux2= self.__definir_punto(opcion_jugador, opcion_maquina)
                puntaje_humano+=aux1
                puntaje_maquina+=aux2

                clear_output()
                print('HUMANO\t\tMAQUINA')
                print(str(puntaje_humano)+'\t\t'+str(puntaje_maquina)+'\n')

                if opcion_jugador==1:
                    print("PIEDRA\t\t", end='')
                elif opcion_jugador==2:
                    print("PAPEL\t\t", end='')
                else:
                    print("TIJERA\t\t", end='')
                
                time.sleep(1.0)

                if opcion_maquina==1:
                    print("PIEDRA", end='')
                elif opcion_maquina==2:
                    print("PAPEL", end='')
                else:
                    print("TIJERA", end='')
                

                opcion_maquina=self.__transformar(opcion_maquina)
                opcion_jugador=self.__transformar(opcion_jugador)


                CSV_row.append(opcion_jugador[0])
                CSV_row.append(opcion_jugador[1])
                CSV_row.append(opcion_jugador[2])

                CSV_row.append(opcion_maquina[0])
                CSV_row.append(opcion_maquina[1])
                CSV_row.append(opcion_maquina[2])

                time.sleep(4.0)
            
            if puntaje_humano>puntaje_maquina:
                print('\n\nGanaste Humano, que tal otro?')
            elif puntaje_humano<puntaje_maquina:
                print('\n\Perdiste Humano, que tal otro?')
            else:
                print('Empate')



            df=pd.DataFrame([CSV_row], columns = self.file_columns)

            df.to_csv(self.file, index=None, mode="a", sep=';', header=False)
            if input('Ganaste Humano, que tal otro? [s]/[n]')=='n':
                break







In [188]:
juego=Cachipun()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [189]:
juego.main()

HUMANO		MAQUINA
1		3

PIEDRA		PAPEL
\Perdiste Humano, que tal otro?
Ganaste Humano, que tal otro? [s]/[n]n


In [103]:
#este metodo tranforma la data historica a un archivo que sera usado por la red neuronal, al hacer esto el archivo historico borra sus registros
juego.processing_file()

In [190]:
#entreno la red neuronal dede el archivo de la data historica
juego.train_ann()


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.900
